In [1]:
import warnings
warnings.filterwarnings("ignore")

### Import Libraries

In [2]:
import pandas as pd
import re
import spacy
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, TensorDataset

### Load Dataset

In [3]:
# Load French Monolingual Dataset
sentences = []

with open('./Dataset/Monolingual/fra_news_2023_1M/fra_news_2023_1M-sentences.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if not line:
            continue

        parts = line.split('\t', 1)
        if len(parts) > 1:
            sentence = parts[1]
            sentences.append(sentence)

french_df = pd.DataFrame(sentences, columns=['Sentence'])
french_df

,Sentence
0,¤ Une coalition (BBY) composée de plus d’une c...
1,$Mais le groupe a aussi annoncé la suppression...
2,"$Toutefois, l’étude menée a également livré de..."
3,"€Dans les locaux d’Atem, à Solliès-Pont, les é..."
4,° 1.000.000 de dirhams hors taxes pour les mar...
...,...
999995,С’est son premier voyage à l’étranger depuis s...
999996,С’est un établissement d'enseignement supérieu...
999997,Сергей ГунеевAccéder à la base multimédiaVladi...
999998,"يا تونِسَ الأُنسِ يا خَضرا المَيادينِ », a écr..."


In [4]:
# Load English Monolingual Dataset
with open('./Dataset/Monolingual/news-commentary-v18.txt', 'r', encoding='utf-8') as file:
    sentences = file.readlines()

sentences = [sentence.strip() for sentence in sentences]
english_df = pd.DataFrame(sentences, columns=['Sentence'])
english_df

,Sentence
0,"$10,000 Gold?"
1,SAN FRANCISCO – It has never been easy to have...
2,"Lately, with gold prices up more than 300% ove..."
3,"Just last December, fellow economists Martin F..."
4,Wouldn’t you know it?
...,...
901593,"At the same time, Zuma’s revolutionary generat..."
901594,"In a region that reveres the elderly, Zuma’s a..."
901595,Three in ten South Africans are younger than 1...
901596,Somehow Zuma must find a way to honor his own ...


In [5]:
x = 20
french_df = french_df[:x]
english_df = english_df[:x]

### Preprocess Data

In [6]:
# Remove empty sentences if any
english_df = english_df[english_df['Sentence'].notna() & (english_df['Sentence'] != '')]
french_df = french_df[french_df['Sentence'].notna() & (french_df['Sentence'] != '')]

In [7]:
def preprocess_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]', '', sentence)  # Remove punctuation
    return sentence

In [8]:
english_df.loc[:, 'Sentence'] = english_df['Sentence'].apply(lambda x: preprocess_text(x))
french_df.loc[:, 'Sentence'] = french_df['Sentence'].apply(lambda x: preprocess_text(x))

In [9]:
def remerge_sent(sent):
    # merges tokens which are not separated by white-space
    # does this recursively until no further changes
    changed = True
    while changed:
        changed = False
        i = 0
        while i < sent.__len__() - 1:
            tok = sent[i]
            if not tok.whitespace_:
                ntok = sent[i + 1]
                # in-place operation.
                with sent.retokenize() as retokenizer:
                    retokenizer.merge(sent[i: i + 2])
                changed = True
            i += 1
    return sent

In [10]:
nlp_en = spacy.load('en_core_web_sm')
nlp_fr = spacy.load('fr_core_news_sm')

In [11]:
def tokenize_with_spacy(sentence, nlp):
    doc = nlp(sentence)
    spacy_sentence = remerge_sent(doc)
    return [token.text for token in spacy_sentence]

# Tokenize
english_df['Tokens'] = english_df['Sentence'].apply(lambda x: tokenize_with_spacy(x, nlp_en))
french_df['Tokens'] = french_df['Sentence'].apply(lambda x: tokenize_with_spacy(x, nlp_fr))

In [12]:
# Drop sentences longer than 50 words
english_df = english_df[english_df['Tokens'].apply(len) <= 50]
french_df = french_df[french_df['Tokens'].apply(len) <= 50]

### Generate BERT Embeddings

We can use pre trained transformer models for generating embeddings for the tokens, we can use different versions of BERT for generation embeddings for French, German etc.

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
# Load the pre-trained BERT tokenizer and model
# Load the fast version of the tokenizer

tokenizer_en = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True)
bert_model_en = AutoModel.from_pretrained('bert-base-uncased')
bert_model_en = bert_model_en.to(device)

tokenizer_fr = AutoTokenizer.from_pretrained('camembert-base', use_fast=True)
bert_model_fr = AutoModel.from_pretrained('camembert-base')
bert_model_fr = bert_model_fr.to(device)

In [15]:
# Retrieves BERT embeddings for a list of tokens using a fast tokenizer, enabling accurate aggregation of subword embeddings into their original token representations.
def get_bert_embeddings(tokens, tokenizer, bert_model):
    inputs = tokenizer(tokens, return_tensors='pt', is_split_into_words=True, padding=False, truncation=True)

    # Get BERT embeddings from the model
    with torch.no_grad():
        outputs = bert_model(**inputs)

    # Get the embeddings for each subword
    token_embeddings = outputs.last_hidden_state.squeeze(0)  # Shape: (sequence_length, hidden_size)
    # Get word_ids to align subword tokens with the original tokens
    word_ids = inputs.word_ids()

    # Aggregate subword embeddings back to their original tokens
    aggregated_embeddings = []
    current_token_embeddings = []

    for idx, word_id in enumerate(word_ids):
        if word_id is None:
            continue
        if len(current_token_embeddings) > 0 and word_id != word_ids[idx - 1]:
            aggregated_embeddings.append(torch.mean(torch.stack(current_token_embeddings), dim=0))
            current_token_embeddings = []
        current_token_embeddings.append(token_embeddings[idx])
    
    if len(current_token_embeddings) > 0:
        aggregated_embeddings.append(torch.mean(torch.stack(current_token_embeddings), dim=0))

    return torch.stack(aggregated_embeddings)

In [16]:
# Function to generate BERT embeddings for dataFrame
def generate_embeddings(df, tokenizer, bert_model):
    embeddings_list = []
    for _, row in df.iterrows():
        tokenized_sentence = row['Tokens']
        embeddings = get_bert_embeddings(tokenized_sentence, tokenizer, bert_model)
        embeddings_list.append(embeddings)
    return embeddings_list

In [17]:
def generate_padded_embeddings(df, embedding_list):
    # Checking the length of longest sentence
    max_len = max(embedding.shape[0] for embedding in embedding_list)
    print(f"The length of the longest sentence is: {max_len}")

    # Pad the embeddings to ensure uniformity across dataset as model will be trained in batches
    padded_embeddings = pad_sequence(embedding_list, batch_first=True)
    df['Embeddings'] = [padded_embeddings[i] for i in range(padded_embeddings.shape[0])]
    print(padded_embeddings.shape)

In [18]:
english_embeddings = generate_embeddings(english_df, tokenizer_en, bert_model_en)
english_embeddings[0].shape

torch.Size([2, 768])

In [19]:
generate_padded_embeddings(english_df, english_embeddings)

The length of the longest sentence is: 31
torch.Size([20, 31, 768])


In [20]:
french_embeddings = generate_embeddings(french_df, tokenizer_fr, bert_model_fr)
french_embeddings[0].shape

torch.Size([30, 768])

In [21]:
generate_padded_embeddings(french_df, french_embeddings)

The length of the longest sentence is: 35
torch.Size([20, 35, 768])


## Denoising Auto Encoding (DAE)

Noise is added to make sure that self auto encoding mechanism is not just returning the same sequence of words, and rather learns the structure of the sentence, otherwise for a random sequence of words also it will return the same output.

In [22]:
def apply_word_dropout(sentence, pwd=0.1, padding_embedding=None):
    noisy_sentence = []
    drop_count = 0

    for word in sentence:
        if random.random() > pwd:
            noisy_sentence.append(word)
        else:
            drop_count += 1

    if padding_embedding is None:
        padding_embedding = torch.zeros_like(sentence[0]) 

    noisy_sentence.extend([padding_embedding] * drop_count)
    noisy_sentence_tensor = torch.stack(noisy_sentence)

    return noisy_sentence_tensor

In [23]:
def apply_sentence_shuffling(sentence, k=3, alpha=0.5):
    n = sentence.size(0)

    # Generating random permutation vector q
    q = torch.arange(n).float() + torch.rand(n) * alpha  # Slightly perturb the indices
    _, permuted_indices = torch.sort(q)  # Sorting to get the permutation

    # Apply the shuffle, respecting the condition |σ(i) - i| <= k
    for i in range(n):
        if abs(permuted_indices[i] - i) > k:
            permuted_indices[i] = i 
    
    # Shuffle sentence according to the permuted indices
    shuffled_sentence = sentence[permuted_indices]
    
    return shuffled_sentence

In [24]:
def apply_noise(x, pwd=0.1, k=3, alpha=0.5):
    if isinstance(x, list):
        x = torch.tensor(x)

    # Apply word dropout
    x_noisy = apply_word_dropout(x, pwd)
    
    # Apply sentence shuffling
    x_noisy = apply_sentence_shuffling(x_noisy, k, alpha)
    
    return x_noisy

#### Encoder Decoder With Attention

This is mainly used for longer sentences. As the sentence gets longer, the context vector $(z)$ passed as the first input to the decoder gets diminished or forgotten, and the decoder loses its attention or focus on the main parts of the sequence, causing a drop in the BLEU score.

#### Decoder

The decoder generates the target sentence by iteratively producing each word $y_i$, based on:

$$
p(y_i \mid y_1, \dots, y_{i-1}, x) = g(y_{i-1}, s_i, c_i)
$$

Where:

- $y_i$ is the target word at time step $i$,
- $s_i$ is the hidden state of the decoder LSTM at time step $i$,
- $c_i$ is the context vector that summarizes the relevant information from the encoder.

---

#### Decoder Hidden State Update

The hidden state of the decoder at each time step $i$ is computed by an RNN / LSTM update. The state is conditioned on the previous hidden state $(s_{i-1})$, the previously generated word $(y_{i-1})$, and the context vector $(c_i)$. The context vector contains information from the encoder and helps the decoder focus on the most relevant parts of the source sentence.

$$
s_i = f(s_{i-1}, y_{i-1}, c_i)
$$

Where:
- $s_i$ is the decoder's hidden state at time step $i$,
- $s_{i-1}$ is the hidden state from the previous time step,
- $y_{i-1}$ is the previously generated target word,
- $c_i$ is the context vector at time step $i$.

---

#### Context Vector

The context vector $c_i$ is a weighted sum of the encoder's hidden states (annotations). It reflects the parts of the source sentence that the decoder should focus on when generating the target word $y_i$. Each hidden state $h_j$ from the encoder is weighted by an attention weight $\alpha_{ij}$, which indicates how much attention to pay to that part of the input sentence at time step $i$.

$$
c_i = \sum_{j=1}^{T_x} \alpha_{ij} h_j
$$

Where:
- $h_j$ is the encoder hidden state at position $j$,
- $\alpha_{ij}$ is the attention weight for position $j$ at decoding time step $i$,
- $T_x$ is the length of the input sequence.

---

#### Attention Weights

The attention weight $\alpha_{ij}$ represents how much focus the decoder should place on the $j$-th hidden state of the encoder when generating the $i$-th target word. These attention weights are computed using a softmax function, which ensures that the weights sum to 1 across all positions in the input sequence. This mechanism allows the decoder to pay varying degrees of attention to different parts of the input sequence at each step.

$$
\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})}
$$

Where:
- $e_{ij}$ is the alignment score for the $j$-th input word and the $i$-th target word,
- The denominator sums over all alignment scores for input positions $k$, normalizing the attention weights.

---

#### Alignment Score

The alignment score $e_{ij}$ is computed by a feedforward neural network, known as the **alignment model**. It measures how well the decoder's previous hidden state $s_{i-1}$ aligns with the encoder's hidden state $h_j$. This score helps decide which parts of the source sentence are most relevant when predicting the next target word.

$$
e_{ij} = a(s_{i-1}, h_j)
$$

Where:
- $s_{i-1}$ is the decoder hidden state at time step $i-1$,
- $h_j$ is the encoder hidden state at position $j$,
- $a$ is the alignment model, which can be a simple feedforward network.

### BiLSTM Encoder

Generates sequence of hidden (latent) states for a given sequence of input embeddings (src/tgt language).

In [25]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True, num_layers=num_layers)
        
        # Fully connected layers for final prediction
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)

        self.activation = nn.ReLU()  # Activation function
        self.dropout = nn.Dropout(0.5)  # Dropout for regularization
        self.batch_norm = nn.BatchNorm1d(hidden_dim * 2)

    def forward(self, inputs):
        lstm_out, _ = self.lstm(inputs)  # (batch_size, seq_len, hidden_dim * 2)
        
        # Apply dropout and batch normalization
        lstm_out = self.dropout(lstm_out)
        lstm_out = self.batch_norm(lstm_out.transpose(1, 2)).transpose(1, 2)
        
        # Pass through fully connected layers
        fc_output = self.activation(self.fc1(lstm_out))  # (batch_size, seq_len, hidden_dim)
        fc_output = self.activation(self.fc2(fc_output))  # (batch_size, seq_len, hidden_dim // 2)
        latent_output = self.fc3(fc_output)  # (batch_size, seq_len, output_dim)
        
        return lstm_out, latent_output

#### LSTM Decoder

Generates sequence of words in a (src/tgt) language based on the previous hidden state of the decoder, the current word (hidden state from the encoder), and a context vector given by a weighted sum over the encoder output states.

In [26]:
class Attention(nn.Module):
    def __init__(self, hidden_dim, latent_dim):
        super(Attention, self).__init__()

        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        # Attention mechanism to compute attention weights
        self.attn = nn.Linear(hidden_dim + latent_dim, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))

    def forward(self, hidden, encoder_outputs):
        """
        Compute the attention weights and context vector.
        
        Args:
            hidden: The hidden state of the decoder (batch_size, hidden_dim)
            encoder_outputs: The outputs from the encoder (batch_size, seq_len, latent_dim)
        
        Returns:
            attn_weights: The attention weights (batch_size, seq_len)
            context: The context vector (batch_size, latent_dim)
        """
        batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)

        # Expand hidden state for concatenation with encoder outputs
        hidden = hidden.unsqueeze(1).expand(batch_size, seq_len, -1)

        # Concatenate hidden state and encoder outputs
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # (batch_size, seq_len, hidden_dim)

        # Compute attention weights using the dot product with v
        energy = energy.transpose(1, 2)  # (batch_size, hidden_dim, seq_len)
        v = self.v.repeat(batch_size, 1).unsqueeze(1)  # (batch_size, 1, hidden_dim)

        # Batched matrix multiplication (bmm)
        attn_weights = torch.bmm(v, energy).squeeze(1)  # (batch_size, seq_len)

        attn_weights = F.softmax(attn_weights, dim=1)  # Normalize attention weights

        # Compute the context vector as a weighted sum of encoder outputs
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)  # (batch_size, latent_dim)

        return context

In [27]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, num_layers=3):
        super(Decoder, self).__init__()
        
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # LSTM layer for decoding
        # Takes input as the context vector of hidden_dim and output from the encoder of latent_dim
        # Outputs a hidden state of hidden_dim
        self.lstm = nn.LSTM(latent_dim + hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)

        # Attention mechanism
        self.attn = Attention(hidden_dim, latent_dim)

        # Fully connected layer to output the prediction
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, latent_vectors, encoder_outputs, hidden):
        """
        Decoder forward pass with attention mechanism.

        Args:
            latent_vectors: The latent vectors from the encoder (batch_size, seq_len, latent_dim), output given by bidirectional LSTM encoder after summarizing (abstracting) the input sentence, that is passing through the fc layers.
            They will be used as input for decoder LSTM, for finding the target word.

            encoder_outputs: The encoder outputs (batch_size, seq_len, latent_dim), output given by bidirectional LSTM encoder of dim 300.
            They will by used for finding the attention weights and context vector.
            
            hidden: The hidden state from the previous time step (num_layers, batch_size, hidden_dim)
            
        Returns:
            outputs: The generated output sequence (batch_size, seq_len, output_dim)
            hidden: The updated hidden state (num_layers, batch_size, hidden_dim)
        """
        batch_size = latent_vectors.size(0)
        seq_len = latent_vectors.size(1)

        # Initialize the output tensor
        outputs = torch.zeros(batch_size, seq_len, self.fc_out.out_features).to(latent_vectors.device)

        # Iterate through the sequence to generate each word
        for t in range(seq_len):
            # Get attention weights and context vector
            # hidden[-1] gives the previous hidden state
            context = self.attn(hidden[-1], encoder_outputs)

            # Concatenate the context vector with the latent vector (decoder input)
            lstm_input = torch.cat((context, latent_vectors[:, t]), dim=1).unsqueeze(1)

            # Pass through the LSTM
            lstm_out, hidden = self.lstm(lstm_input, hidden)  # lstm_out: (batch_size, 1, hidden_dim)
            lstm_out = self.dropout(lstm_out)

            # Predict the next word
            outputs[:, t] = self.fc_out(lstm_out.squeeze(1))

        return outputs, hidden

    def init_hidden(self, batch_size):
        """ Initialize hidden state and cell state for LSTM """
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device)
        return (h0, c0)

### Training the DAE Model

In [28]:
# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 100
NUM_LSTM_LAYERS = 3
LEARNING_RATE = 0.0003

# Encoder
ENCODER_INPUT_DIM = 768 
ENCODER_HIDDEN_DIM = 300
ENCODER_OUTPUT_DIM = 100 

# Decoder
DECODER_OUTPUT_DIM = ENCODER_INPUT_DIM
DECODER_LATENT_DIM = ENCODER_OUTPUT_DIM
DECODER_HIIDEN_DIM = 300

In [29]:
# x and y are same because of auto-encoding
english_dataset = TensorDataset(
    torch.stack(english_df['Embeddings'].tolist()),
    torch.stack(english_df['Embeddings'].tolist()),
)
english_loader = DataLoader(
    english_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

french_dataset = TensorDataset(
    torch.stack(french_df['Embeddings'].tolist()),
    torch.stack(french_df['Embeddings'].tolist()),
)
french_loader = DataLoader(
    french_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [30]:
# Initialize models
encoder_model = Encoder(
    ENCODER_INPUT_DIM,
    ENCODER_HIDDEN_DIM, 
    ENCODER_OUTPUT_DIM,
    NUM_LSTM_LAYERS,
)

decoder_model = Decoder(
    DECODER_LATENT_DIM,
    DECODER_HIIDEN_DIM,
    DECODER_OUTPUT_DIM,
    NUM_LSTM_LAYERS,
)

In [31]:
# Set the models to train mode
encoder_model.train()
decoder_model.train()

Decoder(
  (lstm): LSTM(400, 300, num_layers=3, batch_first=True)
  (attn): Attention(
    (attn): Linear(in_features=400, out_features=300, bias=True)
  )
  (fc_out): Linear(in_features=300, out_features=768, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [32]:
# Initialize optimizer (Adam)
optimizer = optim.Adam(
    list(encoder_model.parameters()) + list(decoder_model.parameters()), 
    lr=LEARNING_RATE,
    betas=(0.5, 0.999),
)

In [33]:
# Loss function (Cosine Loss)
def cosine_similarity_loss(predictions, targets, pad_idx=0): 
    # Flatten the predictions and targets
    predictions = predictions.view(-1, predictions.size(-1))  # (batch_size * seq_len, embedding_dim)
    targets = targets.view(-1, targets.size(-1))  # (batch_size * seq_len, embedding_dim)

    # Create a mask to exclude padded positions
    mask = (targets.sum(dim=-1) != pad_idx).float()  # (batch_size * seq_len,)

    # Compute the cosine similarity between the predictions and targets
    cos_sim = F.cosine_similarity(predictions, targets, dim=-1)  # (batch_size * seq_len,)
    loss = (1 - cos_sim) * mask  # Apply mask to exclude padding positions
    loss = loss.sum() / mask.sum()  # Average the loss over non-padding tokens

    return loss

In [34]:
# Function for training the model
def train_dae(encoder_model, decoder_model, optimizer, data_loader, clip, pad_idx=0):

    for epoch in range(NUM_EPOCHS):
        encoder_model.train()
        decoder_model.train()

        epoch_loss = 0
        for batch in data_loader:
            inputs = batch[0].to(device)  # BERT embeddings
            targets = batch[1].to(device)  # Same because of autoencoding
            
            # Add noise to inputs and stack them into a tensor
            noisy_inputs = torch.stack([apply_noise(sentence) for sentence in inputs]).to(device)

            optimizer.zero_grad()

            # Forward pass through encoder
            encoder_outputs, latent_vectors = encoder_model(noisy_inputs)
            
            # Initialize decoder hidden state for each batch
            hidden, _ = decoder_model.init_hidden(inputs.size(0))  

            # Decoder forward pass with attention
            outputs, hidden = decoder_model(latent_vectors, encoder_outputs, hidden)

            # Compute the loss
            loss = cosine_similarity_loss(outputs, targets, pad_idx)

            # Backward pass and optimization
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(encoder_model.parameters(), clip)
            torch.nn.utils.clip_grad_norm_(decoder_model.parameters(), clip)

            # Optimizer step
            optimizer.step()
            epoch_loss += loss.item()

        # Print the loss for each epoch
        print(f'Epoch [{epoch + 1}/{NUM_EPOCHS}], Loss: {epoch_loss / len(data_loader):.4f}')

In [35]:
# Train DAE for English Dataset
train_dae(
    encoder_model,
    decoder_model,
    optimizer,
    english_loader,
    clip=1.0,
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (620x900 and 400x300)